In [ ]:
from pkgman import include
include(["pandas", "polars", "rich", "tqdm", "requests", "matplotlib", "seaborn", "numpy", "igraph", "json", "pandarallel"])

pd = pandas
pl = polars
tqdm = tqdm
rq = requests
plt = matplotlib.pyplot
sns = seaborn
np = numpy
pp = rich.print
ig = igraph
pdl = pandarallel.pandarallel
pdl.initialize(progress_bar=True, nb_workers=8)

In [ ]:
df_path = "../data/result/users.all.jsonl"
df = pd.read_json(df_path, lines=True)
df.head(5)

In [ ]:
json.loads(df["user_meta"][0])

In [ ]:
# Models created by users
df["models"] = df["user_meta"].parallel_apply(lambda x: json.loads(x)["models"] if x else {})
df["models_count"] = df["models"].parallel_apply(lambda x: len(x))

# Datasets created by users
df["datasets"] = df["user_meta"].parallel_apply(lambda x: json.loads(x)["datasets"] if x else {})
df["datasets_count"] = df["datasets"].parallel_apply(lambda x: len(x))

# User information
df["user_avatar"] = df["user_meta"].parallel_apply(lambda x: json.loads(x)["u"]["avatarUrl"] if x else None)
df["user_pro"] = df["user_meta"].parallel_apply(lambda x: json.loads(x)["u"]["isPro"] if x else None)

# User organizations
df["orgs"] = df["user_meta"].parallel_apply(lambda x: json.loads(x)["u"]["orgs"] if x else None)

# User languages
df["accept_languages"] = df["user_meta"].parallel_apply(lambda x: json.loads(x)["acceptLanguages"] if x else None)
df.to_csv("../data/process/user.info.tsv", index=False, sep="\t")
df.head(5)